In [1]:
from utils import *

In [2]:
data_gen = yield_data()
data = next(data_gen)
series = next(prepare_time_series(data['df'], data['freq']))

In [3]:
series

,T1
1677-09-21 00:12:43.145224193,453.0
1677-09-21 01:12:43.145224193,417.0
1677-09-21 02:12:43.145224193,395.0
1677-09-21 03:12:43.145224193,420.0
1677-09-21 04:12:43.145224193,453.0
...,...
1678-12-18 21:12:43.145224193,83.0
1678-12-18 22:12:43.145224193,135.0
1678-12-18 23:12:43.145224193,132.0
1678-12-19 00:12:43.145224193,158.0


In [4]:
series.columns[0]

'T1'

In [5]:
auto_featurize(series).describe()

,T1,second,minute,hour,day_of_week,week_of_year,half_hourly,month,quarter,year,...,wavelet_1,wavelet_2,wavelet_3,wavelet_4,wavelet_5,wavelet_6,wavelet_7,wavelet_8,micro_fft_peak,micro_fft_amp
count,10898.000000,10898.0,10898.0,10898.000000,10898.000000,10898.0,10898.000000,10898.000000,10898.000000,10898.000000,...,10898.000000,10898.000000,10898.000000,10898.000000,10898.000000,10898.000000,10898.000000,10898.000000,10898.0,10898.000000
mean,58.994403,43.0,12.0,11.497981,3.006056,29.986236,22.995963,7.331253,2.779960,1677.775372,...,0.029216,0.072235,0.124337,0.178046,0.227203,0.268952,0.304311,0.335791,0.0,24.590970
std,62.382589,0.0,0.0,6.923476,1.997557,15.245638,13.846952,3.518905,1.149643,0.417357,...,35.443486,52.973597,68.306754,82.103918,94.572269,106.276999,117.519203,128.613211,0.0,36.882266
min,2.000000,43.0,12.0,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,1677.000000,...,-530.530274,-666.621468,-851.315538,-983.990547,-1087.393778,-1150.724687,-1206.347988,-1241.476422,0.0,0.000000
25%,15.000000,43.0,12.0,5.000000,1.000000,17.0,10.000000,4.000000,2.000000,1678.000000,...,-10.560328,-16.808156,-22.974103,-29.050015,-35.113918,-41.276708,-47.657304,-54.286138,0.0,5.858860
50%,40.000000,43.0,12.0,11.000000,3.000000,33.0,22.000000,8.000000,3.000000,1678.000000,...,-0.643806,-0.989412,-1.674280,-2.562972,-3.191597,-3.988875,-4.774645,-5.373809,0.0,13.513474
75%,84.000000,43.0,12.0,17.000000,5.000000,43.0,34.000000,10.000000,4.000000,1678.000000,...,6.432115,10.183977,14.084528,18.520733,22.624571,26.740119,31.162224,35.743228,0.0,29.125939
max,713.000000,43.0,12.0,23.000000,6.000000,52.0,46.000000,12.000000,4.000000,1678.000000,...,413.556834,560.634184,634.423188,695.508891,718.483229,800.174207,868.487477,925.206785,0.0,551.303521


In [ ]:
from itertools import islice
data_generator = yield_data()
i = 23
next(islice(data_generator, i, i + 1))


{'name': 'dominick_dataset',
 'df':        series_name                                       series_value
 0               T1  [41.83, 0.0, 0.0, 0.0, 41.83, 0.0, 0.0, 0.0, 0...
 1               T2  [68.85, 68.85, 0.0, 68.85, 0.0, 68.85, 0.0, 0....
 2               T3  [0.0, 0.0, 0.0, 62.62, 62.62, 62.62, 0.0, 62.6...
 3               T4  [67.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
 4               T5  [0.0, 0.0, 0.0, 62.58, 62.58, 0.0, 0.0, 62.58,...
 ...            ...                                                ...
 115699     T115700  [0.0, 27.47, 33.54, 33.54, 33.56, 33.67, 33.67...
 115700     T115701  [0.0, 26.31, 31.49, 33.67, 33.67, 33.67, 33.67...
 115701     T115702  [0.0, 26.54, 33.26, 33.26, 33.26, 33.67, 33.67...
 115702     T115703  [0.0, 27.39, 31.48, 33.21, 33.38, 33.67, 33.67...
 115703     T115704  [0.0, 26.89, 0.0, 33.67, 33.67, 33.67, 33.67, ...
 
 [115704 rows x 2 columns],
 'freq': 'weekly'}